## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
# Preview dataframe 
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
clean_application_df = application_df.drop(['EIN','NAME'], axis = 1)
# Preview dataframe 
clean_application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
clean_application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type = clean_application_df['APPLICATION_TYPE'].value_counts()
application_type

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# Set cutoff value
app_type_cutoff = 500
application_type_df = application_type.to_frame()
# Use the variable name `application_types_to_replace`
application_types_to_replace = []
# Create list of records that will be updated
for index in application_type_df.index:
    if application_type_df['APPLICATION_TYPE'].loc[index] < app_type_cutoff:
        application_types_to_replace.append(index)
# Replace in dataframe
for app in application_types_to_replace:
    clean_application_df['APPLICATION_TYPE'] = clean_application_df['APPLICATION_TYPE'].replace(app,"Other")
# Check to make sure binning was successful
clean_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classication_remove = clean_application_df['CLASSIFICATION'].value_counts()
classication_remove

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_one_instance = 1
classication__remove_df = classication_remove.to_frame()
classifications_to_remove = []
# Create list of 'Classifications' that only appeared once
for index in classication__remove_df.index:
    if classication__remove_df['CLASSIFICATION'].loc[index] <= classification_one_instance:
        classifications_to_remove.append(index)
# Remove single instance classifications from the dataframe
for cls1 in classifications_to_remove:
    index_to_remove = clean_application_df[clean_application_df['CLASSIFICATION'] == cls1].index
    clean_application_df.drop(index_to_remove , inplace = True)
clean_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# Set cutoff value
classification_rename_cutoff = 1000
classication_rename = clean_application_df['CLASSIFICATION'].value_counts()
classication_rename_df = classication_rename.to_frame()
# Use the variable name `classifications_to_replace`
classifications_to_replace = []
# Create list of records that will be updated
for index in classication_rename_df.index:
    if classication_rename_df['CLASSIFICATION'].loc[index] <= classification_rename_cutoff:
        classifications_to_replace.append(index)
# Replace in dataframe
for cls2 in classifications_to_replace:
    clean_application_df['CLASSIFICATION'] = clean_application_df['CLASSIFICATION'].replace(cls2,"Other") 
# Check to make sure binning was successful
clean_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2235
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
clean_application_dummies = pd.get_dummies(clean_application_df, drop_first = False)
clean_application_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = clean_application_dummies['IS_SUCCESSFUL'].values
X = clean_application_dummies.drop('IS_SUCCESSFUL', axis = 1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 24)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

## Optimization 01
#### - Activation Function for hidden layers:  Relu
#### - Number of layers: 2
#### - Unit count: 5,5
#### - max Epoch: 100

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_2 = tf.keras.models.Sequential()
# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=5, activation = "relu", input_dim = 43))
# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=5, activation = "relu"))
# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))
# Check the structure of the model
nn_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 220       
                                                                 
 dense_1 (Dense)             (None, 5)                 30        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn_2.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [14]:
# Train the model
fit_model_2 = nn_2.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 1s 580us/step - loss: 0.6269 - accuracy: 0.6775
Epoch 2/100
804/804 [==============================] - 0s 586us/step - loss: 0.5744 - accuracy: 0.7157
Epoch 3/100
804/804 [==============================] - 0s 579us/step - loss: 0.5645 - accuracy: 0.7235
Epoch 4/100
804/804 [==============================] - 0s 585us/step - loss: 0.5612 - accuracy: 0.7248
Epoch 5/100
804/804 [==============================] - 0s 594us/step - loss: 0.5592 - accuracy: 0.7249
Epoch 6/100
804/804 [==============================] - 0s 580us/step - loss: 0.5575 - accuracy: 0.7258
Epoch 7/100
804/804 [==============================] - 0s 578us/step - loss: 0.5567 - accuracy: 0.7293
Epoch 8/100
804/804 [==============================] - 0s 579us/step - loss: 0.5561 - accuracy: 0.7282
Epoch 9/100
804/804 [==============================] - 0s 572us/step - loss: 0.5554 - accuracy: 0.7297
Epoch 10/100
804/804 [==============================] - 0s 584us/step - l

804/804 [==============================] - 1s 695us/step - loss: 0.5487 - accuracy: 0.7336
Epoch 80/100
804/804 [==============================] - 1s 732us/step - loss: 0.5485 - accuracy: 0.7333
Epoch 81/100
804/804 [==============================] - 1s 657us/step - loss: 0.5486 - accuracy: 0.7333
Epoch 82/100
804/804 [==============================] - 1s 679us/step - loss: 0.5487 - accuracy: 0.7333
Epoch 83/100
804/804 [==============================] - 1s 673us/step - loss: 0.5488 - accuracy: 0.7331
Epoch 84/100
804/804 [==============================] - 1s 707us/step - loss: 0.5486 - accuracy: 0.7333
Epoch 85/100
804/804 [==============================] - 1s 651us/step - loss: 0.5485 - accuracy: 0.7332
Epoch 86/100
804/804 [==============================] - 1s 690us/step - loss: 0.5484 - accuracy: 0.7341
Epoch 87/100
804/804 [==============================] - 1s 672us/step - loss: 0.5484 - accuracy: 0.7335
Epoch 88/100
804/804 [==============================] - 1s 730us/step - loss:

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5538 - accuracy: 0.7300 - 283ms/epoch - 1ms/step
Loss: 0.5537996888160706, Accuracy: 0.7299568057060242


In [16]:
# Export our model to HDF5 file
nn_2.save("AlphabetSoupCharity_OPT01.h5")

## Optimization 02
#### - Activation Function for hidden layers:  Tanh
#### - Number of layers: 2
#### - Unit count: 5,5
#### - max Epoch: 100

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_3 = tf.keras.models.Sequential()
# First hidden layer
nn_3.add(tf.keras.layers.Dense(units=5, activation = "tanh", input_dim = 43))
# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=5, activation = "tanh"))
# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))
# Check the structure of the model
nn_3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 5)                 220       
                                                                 
 dense_4 (Dense)             (None, 5)                 30        
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn_3.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [19]:
# Train the model
fit_model_3 = nn_3.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 1s 835us/step - loss: 0.6072 - accuracy: 0.6934
Epoch 2/100
804/804 [==============================] - 1s 835us/step - loss: 0.5715 - accuracy: 0.7239
Epoch 3/100
804/804 [==============================] - 1s 838us/step - loss: 0.5663 - accuracy: 0.7249
Epoch 4/100
804/804 [==============================] - 1s 814us/step - loss: 0.5627 - accuracy: 0.7263
Epoch 5/100
804/804 [==============================] - 1s 833us/step - loss: 0.5610 - accuracy: 0.7273
Epoch 6/100
804/804 [==============================] - 1s 827us/step - loss: 0.5590 - accuracy: 0.7288
Epoch 7/100
804/804 [==============================] - 1s 804us/step - loss: 0.5578 - accuracy: 0.7305
Epoch 8/100
804/804 [==============================] - 1s 813us/step - loss: 0.5563 - accuracy: 0.7299
Epoch 9/100
804/804 [==============================] - 1s 795us/step - loss: 0.5561 - accuracy: 0.7286
Epoch 10/100
804/804 [==============================] - 1s 803us/step - l

804/804 [==============================] - 1s 683us/step - loss: 0.5481 - accuracy: 0.7310
Epoch 80/100
804/804 [==============================] - 1s 673us/step - loss: 0.5482 - accuracy: 0.7301
Epoch 81/100
804/804 [==============================] - 1s 705us/step - loss: 0.5479 - accuracy: 0.7320
Epoch 82/100
804/804 [==============================] - 1s 691us/step - loss: 0.5482 - accuracy: 0.7310
Epoch 83/100
804/804 [==============================] - 1s 684us/step - loss: 0.5480 - accuracy: 0.7314
Epoch 84/100
804/804 [==============================] - 1s 728us/step - loss: 0.5479 - accuracy: 0.7314
Epoch 85/100
804/804 [==============================] - 1s 648us/step - loss: 0.5479 - accuracy: 0.7320
Epoch 86/100
804/804 [==============================] - 1s 693us/step - loss: 0.5481 - accuracy: 0.7312
Epoch 87/100
804/804 [==============================] - 1s 656us/step - loss: 0.5481 - accuracy: 0.7324
Epoch 88/100
804/804 [==============================] - 1s 675us/step - loss:

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5528 - accuracy: 0.7274 - 190ms/epoch - 708us/step
Loss: 0.5528134107589722, Accuracy: 0.7273894548416138


In [21]:
# Export our model to HDF5 file
nn_3.save("AlphabetSoupCharity_OPT02.h5")

## Optimization 03
#### - Activation Function for hidden layers:  Sigmoid (best accuracy score between activation functions)
#### - Number of layers: 2
#### - Unit count: 23, 11
#### - max Epoch: 100

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_4 = tf.keras.models.Sequential()
# First hidden layer
nn_4.add(tf.keras.layers.Dense(units=23, activation = "sigmoid", input_dim = 43))
# Second hidden layer
nn_4.add(tf.keras.layers.Dense(units=11, activation = "sigmoid"))
# Output layer
nn_4.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))
# Check the structure of the model
nn_4.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 23)                1012      
                                                                 
 dense_7 (Dense)             (None, 11)                264       
                                                                 
 dense_8 (Dense)             (None, 1)                 12        
                                                                 
Total params: 1,288
Trainable params: 1,288
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn_4.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [24]:
# Train the model
fit_model_4 = nn_4.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 1s 712us/step - loss: 0.6284 - accuracy: 0.6731
Epoch 2/100
804/804 [==============================] - 1s 861us/step - loss: 0.5725 - accuracy: 0.7245
Epoch 3/100
804/804 [==============================] - 1s 778us/step - loss: 0.5676 - accuracy: 0.7272
Epoch 4/100
804/804 [==============================] - 1s 882us/step - loss: 0.5636 - accuracy: 0.7290
Epoch 5/100
804/804 [==============================] - 1s 816us/step - loss: 0.5609 - accuracy: 0.7281
Epoch 6/100
804/804 [==============================] - 1s 832us/step - loss: 0.5590 - accuracy: 0.7292
Epoch 7/100
804/804 [==============================] - 1s 787us/step - loss: 0.5569 - accuracy: 0.7294
Epoch 8/100
804/804 [==============================] - 1s 839us/step - loss: 0.5561 - accuracy: 0.7302
Epoch 9/100
804/804 [==============================] - 1s 813us/step - loss: 0.5550 - accuracy: 0.7307
Epoch 10/100
804/804 [==============================] - 1s 841us/step - l

804/804 [==============================] - 1s 675us/step - loss: 0.5405 - accuracy: 0.7374
Epoch 80/100
804/804 [==============================] - 1s 681us/step - loss: 0.5405 - accuracy: 0.7379
Epoch 81/100
804/804 [==============================] - 1s 716us/step - loss: 0.5405 - accuracy: 0.7381
Epoch 82/100
804/804 [==============================] - 1s 704us/step - loss: 0.5403 - accuracy: 0.7377
Epoch 83/100
804/804 [==============================] - 1s 651us/step - loss: 0.5404 - accuracy: 0.7378
Epoch 84/100
804/804 [==============================] - 1s 720us/step - loss: 0.5403 - accuracy: 0.7376
Epoch 85/100
804/804 [==============================] - 1s 694us/step - loss: 0.5403 - accuracy: 0.7379
Epoch 86/100
804/804 [==============================] - 1s 681us/step - loss: 0.5402 - accuracy: 0.7379
Epoch 87/100
804/804 [==============================] - 1s 705us/step - loss: 0.5402 - accuracy: 0.7381
Epoch 88/100
804/804 [==============================] - 1s 673us/step - loss:

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5505 - accuracy: 0.7290 - 208ms/epoch - 775us/step
Loss: 0.5504557490348816, Accuracy: 0.7290232181549072


In [26]:
# Export our model to HDF5 file
nn_4.save("AlphabetSoupCharity_OPT03.h5")

## Optimization 04
#### - Activation Function for hidden layers:  Sigmoid (best accuracy score between activation functions)
#### - Number of layers: 3
#### - Unit count: 23, 11, 7
#### - max Epoch: 100

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_5 = tf.keras.models.Sequential()
# First hidden layer
nn_5.add(tf.keras.layers.Dense(units=23, activation = "sigmoid", input_dim = 43))
# Second hidden layer
nn_5.add(tf.keras.layers.Dense(units=11, activation = "sigmoid"))
# Third hidden layer
nn_5.add(tf.keras.layers.Dense(units=7, activation = "sigmoid"))
# Output layer
nn_5.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))
# Check the structure of the model
nn_5.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 23)                1012      
                                                                 
 dense_10 (Dense)            (None, 11)                264       
                                                                 
 dense_11 (Dense)            (None, 7)                 84        
                                                                 
 dense_12 (Dense)            (None, 1)                 8         
                                                                 
Total params: 1,368
Trainable params: 1,368
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model
nn_5.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [29]:
# Train the model
fit_model_5 = nn_5.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 1s 876us/step - loss: 0.6289 - accuracy: 0.6631
Epoch 2/100
804/804 [==============================] - 1s 888us/step - loss: 0.5742 - accuracy: 0.7283
Epoch 3/100
804/804 [==============================] - 1s 902us/step - loss: 0.5730 - accuracy: 0.7299
Epoch 4/100
804/804 [==============================] - 1s 892us/step - loss: 0.5726 - accuracy: 0.7296
Epoch 5/100
804/804 [==============================] - 1s 878us/step - loss: 0.5717 - accuracy: 0.7295
Epoch 6/100
804/804 [==============================] - 1s 891us/step - loss: 0.5710 - accuracy: 0.7306
Epoch 7/100
804/804 [==============================] - 1s 895us/step - loss: 0.5704 - accuracy: 0.7304
Epoch 8/100
804/804 [==============================] - 1s 805us/step - loss: 0.5700 - accuracy: 0.7305
Epoch 9/100
804/804 [==============================] - 1s 868us/step - loss: 0.5693 - accuracy: 0.7314
Epoch 10/100
804/804 [==============================] - 1s 884us/step - l

804/804 [==============================] - 1s 775us/step - loss: 0.5413 - accuracy: 0.7364
Epoch 80/100
804/804 [==============================] - 1s 675us/step - loss: 0.5414 - accuracy: 0.7365
Epoch 81/100
804/804 [==============================] - 1s 742us/step - loss: 0.5412 - accuracy: 0.7373
Epoch 82/100
804/804 [==============================] - 1s 732us/step - loss: 0.5410 - accuracy: 0.7369
Epoch 83/100
804/804 [==============================] - 1s 726us/step - loss: 0.5412 - accuracy: 0.7369
Epoch 84/100
804/804 [==============================] - 1s 728us/step - loss: 0.5410 - accuracy: 0.7368
Epoch 85/100
804/804 [==============================] - 1s 745us/step - loss: 0.5410 - accuracy: 0.7365
Epoch 86/100
804/804 [==============================] - 1s 692us/step - loss: 0.5409 - accuracy: 0.7364
Epoch 87/100
804/804 [==============================] - 1s 724us/step - loss: 0.5409 - accuracy: 0.7370
Epoch 88/100
804/804 [==============================] - 1s 722us/step - loss:

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_5.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5501 - accuracy: 0.7290 - 198ms/epoch - 738us/step
Loss: 0.5500869750976562, Accuracy: 0.7290232181549072


In [31]:
# Export our model to HDF5 file
nn_5.save("AlphabetSoupCharity_OPT04.h5")

## Optimization 05
#### - Ran hyperparameters

In [32]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_6 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_6.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=23,
        step=2), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_6.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=23,
            step=2),
            activation=activation))
    
    nn_6.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_6.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_6

In [33]:
# Import the kerastuner library
import kerastuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

C:\Users\tomsa\AppData\Local\Temp/ipykernel_14236/2980860635.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [34]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 21s]
val_accuracy: 0.7308903932571411

Best val_accuracy So Far: 0.7318240404129028
Total elapsed time: 00h 08m 11s
INFO:tensorflow:Oracle triggered exit


In [35]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 11,
 'num_layers': 3,
 'units_0': 21,
 'units_1': 21,
 'units_2': 9,
 'units_3': 17,
 'units_4': 1,
 'units_5': 7,
 'units_6': 7,
 'units_7': 3,
 'units_8': 3,
 'units_9': 5,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [36]:
# Evaluate best model against test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5491 - accuracy: 0.7318 - 291ms/epoch - 1ms/step
Loss: 0.5490672588348389, Accuracy: 0.7318240404129028


In [37]:
# Export our model to HDF5 file
best_model.save("AlphabetSoupCharity_OPT05.h5")